# CHIRPS rainfaill processing

Process daily rainfall hosted on Google Earth Engine. 

In [1]:
!pip install geopandas intake intake-stac eefolium xarray geoviews 

## Authentification

### Authentification to GEE

In [2]:
from google.colab import files
files.upload()

Saving aoi.shx to aoi (1).shx
Saving aoi.shp to aoi (1).shp
Saving aoi.dbf to aoi (1).dbf
Saving aoi.qpj to aoi (1).qpj
Saving aoi.prj to aoi (1).prj


{'aoi.dbf': b'\x03x\t\x15\x01\x00\x00\x00A\x00\x0b\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00id\x00\x00\x00\x00\x00\x00\x00\x00\x00N\x00\x00\x00\x00\n\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\r          1',
 'aoi.prj': b'GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137,298.257223563]],PRIMEM["Greenwich",0],UNIT["Degree",0.017453292519943295]]',
 'aoi.qpj': b'GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4326"]]\n',
 'aoi.shp': b"\x00\x00'\n\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xae\xe8\x03\x00\x00\x05\x00\x00\x00|`\x84\xca\x8a\xfdY@wj\xc1\x86\xf0\xa4D@\x197\xa3A\n\xdd[@\x89\xafw\xb8ucF@\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x

In [3]:
import ee

In [4]:
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=bJrlRVuQdxafPNhr6HqbpNG1z5bdtRz8WJblOKf444A&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AY0e-g6MFBV4XbGJzrA5hXLTttCeN9X1D6J-82MSRAoQbuq7Gi8t7ry8CIk

Successfully saved authorization token.


### Authentification to Drive

We will use an export to Google Drive since this seems like the most stable option. 

In [48]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## CHIRPS 

[CHIRPS](https://www.chc.ucsb.edu/data/chirps) data is hosted (not uniquely) on Google Earth Engine. 

[chirps_gee](https://developers.google.com/earth-engine/datasets/catalog/UCSB-CHG_CHIRPS_DAILY)

As always with large data analysis we want to bring the compute to the data which is exactly one of the features of GEE. 
So most of the analysis will be performed on GEE and result downloaded. 

In [53]:
gee_collection_id = "UCSB-CHG/CHIRPS/DAILY"
band_name = "precipitation"
aoi_url = 'https://github.com/matthieu-bernard/gee-processing/raw/main/data/aoi.geojson'


## Region of Interest

## Read region of interest vector file

In [54]:
import geopandas as gpd

gpd_aoi = gpd.GeoDataFrame.from_file(aoi_url)
gjson = gpd_aoi.to_crs(epsg='4326').to_json()

In [22]:
# Really simple plot of the region of interest. 
# Just for the sake of control. 

import folium
import shapely
from typing import Tuple

def centroid(geom: shapely.geometry) -> Tuple[float, float]:
    centroid = geom.centroid
    return [centroid.y, centroid.x]


aoi_polygon = gpd_aoi['geometry'][0]
env = aoi_polygon.envelope 

mapa = folium.Map(centroid(aoi_polygon),
                  zoom_start=5,
                  tiles='cartodbpositron')

aoi = folium.features.GeoJson(gjson)
bbox = folium.features.GeoJson(env)

mapa.add_children(aoi)
folium.features.GeoJson(env).add_to(mapa)
mapa



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: FutureWarning: Method `add_children` is deprecated. Please use `add_child` instead.


In [23]:
aoi_polygon.area

12.872835408149841

In [24]:
print(env)

POLYGON ((103.9615961354011 41.28859028286109, 111.4537510007762 41.28859028286109, 111.4537510007762 44.77703004688289, 103.9615961354011 44.77703004688289, 103.9615961354011 41.28859028286109))


## Upload area of interets to GEE

There seems to be 2 options here:
  - Manual upload from the GUI tool: https://developers.google.com/earth-engine/guides/importing
  - Use the cli to upload from GCS (cli is written in python so no need for a external call though https://github.com/google/earthengine-api/blob/7a6f605e1be30002c4a3ec516669703dea9e3f71/python/ee/cli/commands.py). 
  
Both have drawbacks, either need a Google Cloud account or manual intervention in the pipeline. 

To avoid that we can either just filter with the bounding box or create the ee.Geometry on the fly. It's more flexible although it definitely have a the obvious drawback that we would send the geometry definiton over the network each time we run the pipeline. 


In [25]:
import os
import json

def geojson_to_ee(geo_json: str, geodesic=True) -> ee.Geometry:
    """Converts a geojson to ee.Geometry()
    Args:
        geo_json (dict): A geojson geometry dictionary or file path.
    Returns:
        ee_object: An ee.Geometry object
    """

    try:

        geo_json = json.loads(geo_json)
        if geo_json['type'] == 'FeatureCollection':
            features = ee.FeatureCollection(geo_json['features'])
            return features
        elif geo_json['type'] == 'Feature':
            geom = None
            keys = geo_json['properties']['style'].keys()
            if 'radius' in keys:  # Checks whether it is a circle
                geom = ee.Geometry(geo_json['geometry'])
                radius = geo_json['properties']['style']['radius']
                geom = geom.buffer(radius)
            elif geo_json['geometry']['type'] == 'Point':  # Checks whether it is a point
                coordinates = geo_json['geometry']['coordinates']
                longitude = coordinates[0]
                
                latitude = coordinates[1]
                geom = ee.Geometry.Point(longitude, latitude)
            else:
                geom = ee.Geometry(geo_json['geometry'], "", geodesic)
            return geom
        else:
            print("Could not convert the geojson to ee.Geometry()")

    except Exception as e:
        print("Could not convert the geojson to ee.Geometry()")
        print(e)

In [26]:
aoi = geojson_to_ee(gjson)

# read the STAC metadata

Google Actually publish SpatioTemporal Catalog Assets for the catalogs they host in GEE.

For now it seems like processing of the data with a Open Source platform (like Pangeo) is not there yet due to the 'proprietary' storage format they use. But lets at least salute the effort of publishing metadata to the STAC standard. 

[CHIRPS_stac_metadata](https://gee.stac.cloud/3cRrouu75vEzeP4uufqKgwzeubDQYizmMUQm?t=bands)

In [27]:
import intake

url = 'https://earthengine-stac.storage.googleapis.com/catalog/UCSB-CHG_CHIRPS_DAILY.json'
cat = intake.open_stac_catalog(url)
cat.metadata

{'description': 'Climate Hazards Group InfraRed Precipitation with Station data (CHIRPS)\nis a 30+ year quasi-global rainfall dataset. CHIRPS incorporates\n0.05° resolution satellite imagery with in-situ station data\nto create gridded rainfall time series for trend analysis and seasonal\ndrought monitoring.\n',
 'extent': {'spatial': {'bbox': [[-180.0, -50.0, 180.0, 50.0]]},
  'temporal': {'interval': [['1981-01-01T00:00:00Z', None]]}},
 'gee:type': 'image_collection',
 'id': 'UCSB-CHG/CHIRPS/DAILY',
 'keywords': ['chg',
  'climate',
  'geophysical',
  'precipitation',
  'ucsb',
  'weather'],
 'license': 'proprietary',
 'properties': {'eo:bands': [{'description': 'Precipitation',
    'gee:unit': 'mm/day',
    'name': 'precipitation'}],
  'gee:cadence': 'day',
  'gee:terms_of_use': 'This datasets are in the public domain. To the extent possible under law,\n[Pete Peterson](https://chc.ucsb.edu/people/pete-peterson)\nhas waived all copyright and related or neighboring rights to\nClimate 

In [28]:
aoi_shp = ee.FeatureCollection('users/matthieu_bernard/aoi').geometry()


In [29]:
import datetime
import dateutil.parser
# Here we get the extent of the datastet to get all of it expect the data for the current 
# since a partial month could pollute or monthly statistics 


start = dateutil.parser.isoparse(cat.metadata['extent']['temporal']['interval'][0][0])
stop = datetime.date.today().replace(day=1)

# Select dataset of interest 
precip = ee.ImageCollection(gee_collection_id).select(band_name)

In [30]:
precip = (ee.ImageCollection(gee_collection_id)
          .select(band_name)
          .filter(ee.Filter.date(start.isoformat(), stop.isoformat()))  
         )

In [31]:
# Convert from daily precipitation values bin (rainy/not-rainy). 
precip_event = precip.map(lambda img: img.gt(0).copyProperties(img, img.propertyNames()))

In [36]:
# get sum per month
months = ee.List.sequence(1, 12)
years = ee.List.sequence(start.year, stop.year)

by_month_year = ee.ImageCollection.fromImages(
  years.map(lambda y: months.map(lambda m:
        (precip_event
        .filter(ee.Filter.calendarRange(y, y, 'year'))
        .filter(ee.Filter.calendarRange(m, m, 'month'))
        .sum()
        .set('month', m).set('year', y))
  )
).flatten())

In [37]:
by_month = ee.ImageCollection.fromImages(
  months.map(lambda m:
        (by_month_year
        .filterMetadata('month', 'equals', m)
        .mean()
        .clip(aoi)
        .set('month', m))
  ))

In [38]:
images = byMonth.toBands()

task = ee.batch.Export.image(images, 'monthly_average_rainy_days_py_100',
  {
  'scale': 5000,
  'maxPixels': 1.0E13,
  'region': aoi_shp,
  'fileFormat': 'GeoTIFF',
  'formatOptions': {
    'cloudOptimized': True
  },
  'folder': 'chirps'
})

#task.start()

In [39]:
latlon = ee.Image.pixelLonLat().addBands(byMonth)


In [44]:
# apply reducer to list
latlon = latlon.reduceRegion(
  reducer=ee.Reducer.toList(),
  geometry=aoi,
  maxPixels=1e8,
  scale=5000)



In [45]:
import numpy as np 


In [47]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Mounted at /content/drive
